# OOI Equipment mapping
- by Landung Setiawan
- 5/31/2016
- This notebook is for retrieving information from google sheets and then mapping to a JSON file, each instrument has its own JSON file configuration
- The required libraries for this manipulation is *gspread*, *oauth2client*, and *pycrypto*

In [1]:
# Google Authentication Libraries
import oauth2client, gspread
import json

# oauth2client version check and gspread
oauth_ver = oauth2client.__version__
gspread_ver = gspread.__version__

print "oauth2client version : {}".format(oauth_ver) 
print "gspread version : {}".format(gspread_ver)

oauth2client version : 1.5.2
gspread version : 0.3.0


In [2]:
if oauth_ver < "2.0.2":
    from oauth2client.client import SignedJwtAssertionCredentials

    json_key = json.load(open('Nanoos-fcdeeb760f83.json'))
    # Get scope for google sheets
    # Gather all spreadsheets shared with the client_email: nanoos-2016@appspot.gserviceaccount.com
    scope = ['https://spreadsheets.google.com/feeds']
    
    # Retrieve credentials from JSON key of service account
    credentials = SignedJwtAssertionCredentials(json_key['client_email'], json_key['private_key'], scope)
    
    # Authorize gspread to connect to google sheets
    gc = gspread.authorize(credentials)
else:
    from oauth2client.service_account import ServiceAccountCredentials
    # Get scope for google sheets
    # Gather all spreadsheets shared with the client_email: nanoos-2016@appspot.gserviceaccount.com
    scope = ['https://spreadsheets.google.com/feeds']

    # Retrieve credentials from JSON key of service account
    credentials = ServiceAccountCredentials.from_json_keyfile_name('Nanoos-fcdeeb760f83.json', scope)

    # Authorize gspread to connect to google sheets
    gc = gspread.authorize(credentials)

In [3]:
# Get all spreadsheets available for NANOOS
gsheets = gc.openall()
# Get title of the spreadsheets
for i in range(0,len(gsheets)):
    print "{0} {1}".format(i,gsheets[i].title)

0 2016_Spring_NEMO
1 ooi_equipment
2 sensor_configurations_mappings
3 GOA-ON_structering
4 AGGI_Table
5 WINTER 2014 SCHEDULE
6 ocean_extents
7 nanoos_asset_list_20160427T092836
8 nanoos_asset_list_20160429T083128


In [4]:
# Open sensor_configurations_mappings only
sc = gc.open("sensor_configurations_mappings")

In [5]:
# Get all worksheets in a sheet
wks = sc.worksheets()
wks

[<Worksheet 'instruments' id:o5yzc1h>, <Worksheet 'measurements' id:odfoenj>]

In [6]:
s1 = sc.get_worksheet(0)
s2 = sc.get_worksheet(1)
print s1, s2

<Worksheet 'instruments' id:o5yzc1h> <Worksheet 'measurements' id:odfoenj>


## Parsing data to a pandas dataframe
- Now that connection has been established, data is parsed to be viewed

In [7]:
# Import pandas and numpy to make data easier to view
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
print "pandas version: {}".format(pd.__version__)
print "numpy version: {}".format(np.__version__)

/home/lsetiawan/anaconda2/envs/OOI/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


pandas version: 0.18.1
numpy version: 1.10.0


In [8]:
# Getting all the values of sheet1
array = s2.get_all_values()
array

[['instrument',
  'data_products',
  'relative_depth_m',
  'OOI_units',
  'measurement_label'],
 ['metbk', 'air_temperature', '4.1', 'degC', 'A1_AirTemp'],
 ['metbk', 'barometric_pressure', '4.3', 'mbar', 'A1_BarPress'],
 ['metbk', 'relative_humidity', '4.1', '%', 'A1_RelHumidity'],
 ['metbk', 'eastward_wind_velocity', '4.7', 'm/s', 'A1_WindSpeed'],
 ['metbk', 'northward_wind_velocity', '4.7', 'm/s', 'A1_WindSpeed'],
 ['metbk', 'longwave_irradiance', '4.6', 'W/m2', 'A1_SolarRad'],
 ['metbk', 'shortwave_irradiance', '4.6', 'W/m2', 'A1_SolarRad'],
 ['metbk', 'precipitation_level', '4.1', 'mm', 'A2_Rain'],
 ['metbk', 'sea_surface_conductivity', '-1.0', 'S/m', 'H2_Conductiv'],
 ['metbk', 'sea_surface_temperature', '-1.0', 'degC', 'H1_WaterTemp'],
 ['ctdbp', 'temperature', '0', 'degC', 'H1_WaterTemp'],
 ['ctdbp', 'conductivity', '0', 'S/m', 'H2_Conductiv'],
 ['ctdbp', 'pressure', '0', 'Pa', 'H1_Pressure'],
 ['ctdbp', 'oxygen_concentration', '0', '', ''],
 ['wavss', 'peak_period_read', '0', 

In [9]:
# Convert data into pandas dataframe
df = pd.DataFrame(array)
df.columns = array[0]
df.drop(df.index[0], inplace=True)
df = df.convert_objects(convert_numeric=True)
df

/home/lsetiawan/anaconda2/envs/OOI/lib/python2.7/site-packages/ipykernel/__main__.py:5: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


,instrument,data_products,relative_depth_m,OOI_units,measurement_label
1,metbk,air_temperature,4.1,degC,A1_AirTemp
2,metbk,barometric_pressure,4.3,mbar,A1_BarPress
3,metbk,relative_humidity,4.1,%,A1_RelHumidity
4,metbk,eastward_wind_velocity,4.7,m/s,A1_WindSpeed
5,metbk,northward_wind_velocity,4.7,m/s,A1_WindSpeed
6,metbk,longwave_irradiance,4.6,W/m2,A1_SolarRad
7,metbk,shortwave_irradiance,4.6,W/m2,A1_SolarRad
8,metbk,precipitation_level,4.1,mm,A2_Rain
9,metbk,sea_surface_conductivity,-1.0,S/m,H2_Conductiv
10,metbk,sea_surface_temperature,-1.0,degC,H1_WaterTemp


In [10]:
# retrieve only metbk instrument
metbk = df.loc[df['instrument'] == 'metbk']

In [11]:
metbk

,instrument,data_products,relative_depth_m,OOI_units,measurement_label
1,metbk,air_temperature,4.1,degC,A1_AirTemp
2,metbk,barometric_pressure,4.3,mbar,A1_BarPress
3,metbk,relative_humidity,4.1,%,A1_RelHumidity
4,metbk,eastward_wind_velocity,4.7,m/s,A1_WindSpeed
5,metbk,northward_wind_velocity,4.7,m/s,A1_WindSpeed
6,metbk,longwave_irradiance,4.6,W/m2,A1_SolarRad
7,metbk,shortwave_irradiance,4.6,W/m2,A1_SolarRad
8,metbk,precipitation_level,4.1,mm,A2_Rain
9,metbk,sea_surface_conductivity,-1.0,S/m,H2_Conductiv
10,metbk,sea_surface_temperature,-1.0,degC,H1_WaterTemp


In [12]:
# get values of the first row as a list
print metbk.iloc[0].values

['metbk' 'air_temperature' 4.0999999999999996 'degC' 'A1_AirTemp']


In [13]:
len(metbk)

10

In [14]:
def create_JSON(instrument,data_products,relative_depth,OOI_units,measurement_label):
    # Create main JSON keys
    mainkey = dict()
    if instrument != "":
        mainkey["instrument"] = instrument
    if data_products != "":
        mainkey["data_products"] = data_products
    if relative_depth != "":
        mainkey["relative_depth_m"] = relative_depth
    if OOI_units != "":
        mainkey["OOI_units"] = OOI_units
    if measurement_label != "":
        mainkey["measurement_label"] = measurement_label
        
    return mainkey

In [15]:
# get list of dictionary for each of the data products
dict_list = []
for i in range(0,len(metbk)):
    print metbk.iloc[i].values
    metbk_json = create_JSON(metbk.iloc[i].values[0],metbk.iloc[i].values[1],
                             metbk.iloc[i].values[2],metbk.iloc[i].values[3],metbk.iloc[i].values[4])
    dict_list.append(metbk_json)

['metbk' 'air_temperature' 4.0999999999999996 'degC' 'A1_AirTemp']
['metbk' 'barometric_pressure' 4.2999999999999998 'mbar' 'A1_BarPress']
['metbk' 'relative_humidity' 4.0999999999999996 '%' 'A1_RelHumidity']
['metbk' 'eastward_wind_velocity' 4.7000000000000002 'm/s' 'A1_WindSpeed']
['metbk' 'northward_wind_velocity' 4.7000000000000002 'm/s' 'A1_WindSpeed']
['metbk' 'longwave_irradiance' 4.5999999999999996 'W/m2' 'A1_SolarRad']
['metbk' 'shortwave_irradiance' 4.5999999999999996 'W/m2' 'A1_SolarRad']
['metbk' 'precipitation_level' 4.0999999999999996 'mm' 'A2_Rain']
['metbk' 'sea_surface_conductivity' -1.0 'S/m' 'H2_Conductiv']
['metbk' 'sea_surface_temperature' -1.0 'degC' 'H1_WaterTemp']


In [16]:
# prints the dictionary for JSON
print dict_list

[{'instrument': 'metbk', 'data_products': 'air_temperature', 'measurement_label': 'A1_AirTemp', 'OOI_units': 'degC', 'relative_depth_m': 4.0999999999999996}, {'instrument': 'metbk', 'data_products': 'barometric_pressure', 'measurement_label': 'A1_BarPress', 'OOI_units': 'mbar', 'relative_depth_m': 4.2999999999999998}, {'instrument': 'metbk', 'data_products': 'relative_humidity', 'measurement_label': 'A1_RelHumidity', 'OOI_units': '%', 'relative_depth_m': 4.0999999999999996}, {'instrument': 'metbk', 'data_products': 'eastward_wind_velocity', 'measurement_label': 'A1_WindSpeed', 'OOI_units': 'm/s', 'relative_depth_m': 4.7000000000000002}, {'instrument': 'metbk', 'data_products': 'northward_wind_velocity', 'measurement_label': 'A1_WindSpeed', 'OOI_units': 'm/s', 'relative_depth_m': 4.7000000000000002}, {'instrument': 'metbk', 'data_products': 'longwave_irradiance', 'measurement_label': 'A1_SolarRad', 'OOI_units': 'W/m2', 'relative_depth_m': 4.5999999999999996}, {'instrument': 'metbk', 'da

In [17]:
# prints the JSON structured dictionary
print json.dumps(dict_list, sort_keys=False, indent=4, separators=(',', ': '))

[
    {
        "instrument": "metbk",
        "data_products": "air_temperature",
        "measurement_label": "A1_AirTemp",
        "OOI_units": "degC",
        "relative_depth_m": 4.0999999999999996
    },
    {
        "instrument": "metbk",
        "data_products": "barometric_pressure",
        "measurement_label": "A1_BarPress",
        "OOI_units": "mbar",
        "relative_depth_m": 4.2999999999999998
    },
    {
        "instrument": "metbk",
        "data_products": "relative_humidity",
        "measurement_label": "A1_RelHumidity",
        "OOI_units": "%",
        "relative_depth_m": 4.0999999999999996
    },
    {
        "instrument": "metbk",
        "data_products": "eastward_wind_velocity",
        "measurement_label": "A1_WindSpeed",
        "OOI_units": "m/s",
        "relative_depth_m": 4.7000000000000002
    },
    {
        "instrument": "metbk",
        "data_products": "northward_wind_velocity",
        "measurement_label": "A1_WindSpeed",
        "OOI_units":

In [18]:
# Output to JSON file called metbk.json
fj = open("metbk.json", 'w')
fj.write(json.dumps(dict_list, sort_keys=False, indent=4, separators=(',', ': ')))
fj.close()

In [19]:
# retrieve only wavss instrument
wavss = df.loc[df['instrument'] == 'wavss']
wavss

,instrument,data_products,relative_depth_m,OOI_units,measurement_label
15,wavss,peak_period_read,0.0,s,
16,wavss,average_tenth_height,0.0,m,
17,wavss,significant_wave_height,0.0,m,
18,wavss,mean_spectral_period,0.0,s,
19,wavss,spectral_wave_height,0.0,m,
20,wavss,mean_directional_spread,0.0,deg,
21,wavss,peak_period,0.0,s,
22,wavss,significant_wave_period,0.0,s,H1_DomWavePrd
23,wavss,mean_wave_direction,0.0,deg,H1_WaveMeanDir
24,wavss,average_tenth_period,0.0,s,


In [20]:
# get list of dictionary for each of the data products
dict_list = []
for i in range(0,len(wavss)):
    print wavss.iloc[i].values
    wavss_json = create_JSON(wavss.iloc[i].values[0],wavss.iloc[i].values[1],
                             wavss.iloc[i].values[2],wavss.iloc[i].values[3],wavss.iloc[i].values[4])
    dict_list.append(wavss_json)

['wavss' 'peak_period_read' 0.0 's' '']
['wavss' 'average_tenth_height' 0.0 'm' '']
['wavss' 'significant_wave_height' 0.0 'm' '']
['wavss' 'mean_spectral_period' 0.0 's' '']
['wavss' 'spectral_wave_height' 0.0 'm' '']
['wavss' 'mean_directional_spread' 0.0 'deg' '']
['wavss' 'peak_period' 0.0 's' '']
['wavss' 'significant_wave_period' 0.0 's' 'H1_DomWavePrd']
['wavss' 'mean_wave_direction' 0.0 'deg' 'H1_WaveMeanDir']
['wavss' 'average_tenth_period' 0.0 's' '']
['wavss' 'average_wave_height' 0.0 'm' 'H1_WaveHght']
['wavss' 'average_wave_period' 0.0 's' '']
['wavss' 'num_zero_crossings' 0.0 '' '']
['wavss' 'maximum_wave_height' 0.0 'm' '']


In [21]:
# prints the JSON structured dictionary
print json.dumps(dict_list, sort_keys=False, indent=4, separators=(',', ': '))

[
    {
        "instrument": "wavss",
        "data_products": "peak_period_read",
        "OOI_units": "s",
        "relative_depth_m": 0.0
    },
    {
        "instrument": "wavss",
        "data_products": "average_tenth_height",
        "OOI_units": "m",
        "relative_depth_m": 0.0
    },
    {
        "instrument": "wavss",
        "data_products": "significant_wave_height",
        "OOI_units": "m",
        "relative_depth_m": 0.0
    },
    {
        "instrument": "wavss",
        "data_products": "mean_spectral_period",
        "OOI_units": "s",
        "relative_depth_m": 0.0
    },
    {
        "instrument": "wavss",
        "data_products": "spectral_wave_height",
        "OOI_units": "m",
        "relative_depth_m": 0.0
    },
    {
        "instrument": "wavss",
        "data_products": "mean_directional_spread",
        "OOI_units": "deg",
        "relative_depth_m": 0.0
    },
    {
        "instrument": "wavss",
        "data_products": "peak_period",
        "OO

In [22]:
# Output to JSON file called metbk.json
fj = open("wavss.json", 'w')
fj.write(json.dumps(dict_list, sort_keys=False, indent=4, separators=(',', ': ')))
fj.close()